## Projeto automação Web - Busca de desconto em produtos e Notificação por e-mail:

In [13]:
'''Lógica do programa:
-Importar planilha para o Python;
-Percorrer a planilha;
    -Para cada linha:
            -Pegar preço Amazon;
            -Pegar preço LAME;
            -Pegar preço MGLU;
            -Registrar o menor preço;
            -Registrar o local do menor preço;
            -Verificar a necessidade de envio de e-mail ou não(desconto de 20% ou mais sobre o valor original).
-Salvar a planilha.
'''

#Importando Bibliotecas:
import pandas as pd
from selenium import webdriver
import win32com.client as win32

#Acessando os links em excel e criando a tabela:
produtos = pd.read_excel(r'Produtos.xlsx')
produtos = produtos.fillna('-')
display(produtos)



,Unnamed: 0,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,3999.00,Lojas Americanas
1,1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/Smart-LG-Intelig%C3%...,https://www.americanas.com.br/produto/49516150...,https://www.magazineluiza.com.br/smart-tv-uhd-...,2900,2564.05,Magazine Luiza


In [14]:
#Criando uma função de tratamento de texto:
def transformar_texto(texto):
    return float(texto.replace('R$', '').replace('.', '').replace(',', '.'))

In [19]:
#Abrindo o Chrome 'Escondido':
# Esta seria a maneira correta, com o ChromeOptions() e o .add_argument('--headless'), 
#porém alguns sites não funcionam bem com ele:
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')

#Foi utilizado o comando driver.set_window_position(-10000,0), para que posicione o navegador fora da tela, pois
#o parâmetro ChromeOptions('--headless') não funciona para todas as páginas:
driver = webdriver.Chrome()
driver.set_window_position(-10000,0)

enviar_email = False
desconto_min = 0.2

#Automatizando o código para todos os produtos e links:
for i, linha in produtos.iterrows():
    #Pegar preço Amazon:
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element_by_class_name('a-size-base.a-color-price').text
        preco_amazon = transformar_texto(preco_amazon)
    except:
        print('Produto {} não disponível na Amazon'.format(linha['Link Produto']))
        preco_amazon = linha['Preço Original'] * 3

    #Pegar preço LAME:
    driver.get(linha['Lojas Americanas'])
    try:
        preco_lame = driver.find_element_by_class_name('src__BestPrice-sc-1jvw02c-5.cBWOIB.priceSales').text
        preco_lame = transformar_texto(preco_lame)
    except:
        print('Produto {} não disponível nas Lojas Americanas'.format(linha['Link Produto']))      
        preco_lame = linha['Preço Original'] * 3
        
    #Pegar preço MGLU:
    driver.get(linha['Magazine Luiza'])
    try:
        preco_mglu = driver.find_element_by_class_name('price-template__text').text
        preco_mglu = transformar_texto(preco_mglu)
    except:
        print('Produto {} não disponível na Magazine Luiza'.format(linha['Link Produto']))
        preco_mglu = linha['Preço Original'] * 3
        
    preco_original = linha['Preço Original']
    
    #Criando uma lista com os produtos para comparar os preços:
    lista_precos = [(preco_amazon, 'Amazon'), (preco_lame, 'Lojas Americanas'), (preco_mglu, 'Magazine Luiza'), (preco_original, 'Original')]
    
    #Ordenando os preços do menor para o maior:
    lista_precos.sort()
        
    #Preenchendo os dados de preço atual e local:
    produtos.loc[i, 'Preço Atual'] = lista_precos[0][0]
    produtos.loc[i, 'Local'] = lista_precos[0][1]
    
    #Criando a condição para envio do email:
    if lista_precos[0][0] <= preco_original*(1-desconto_min):
        enviar_email = True

#Salvar arquivo:
produtos.to_excel('Produtos.xlsx')

#Enviar e-mail, caso o desconto do produto for igual ou acima de 20%:
outlook = win32.Dispatch('outlook.application')
if enviar_email:
    #enviar e-mail:
    mail = outlook.CreateItem(0)  #Este cria o e-mail
    mail.To = 'carvalhol.amt+compras@gmail.com'
    mail.Subject = f'Produto(s) Encontrado(s) com mais de {desconto_min:.0%} de Desconto'
    
    #Filtrar a tabela de produtos:
    tabela_filtrada = produtos.loc[produtos['Preço Atual'] <= produtos['Preço Original']*(1-desconto_min), :]
    
    mail.HTMLBody = f'<p>Estes são os produtos com mais de {desconto_min:.0%} de desconto:</p>{tabela_filtrada.to_html()}'
    # Não haverá anexos, pois o link do produto irá no corpo do e-mail.
    mail.Send()

#Finalizando o código:
print('Fim da Análise.')
#Fechando o navegador:
driver.quit()
    

Fim da Análise.
